In [ ]:
!pip install pygame 

import pygame
import math
import random
import sys

# Initialize pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 1000, 700
FPS = 60
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (100, 100, 255)
LIGHT_BLUE = (150, 150, 255)
BRIGHT_BLUE = (80, 160, 255)
GRAY = (100, 100, 100)
YELLOW = (255, 255, 0)
RED = (255, 0, 0)
DARK_GRAY = (20, 20, 30)
PANEL_BG = (30, 30, 40, 220)

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("CHARM: Cherenkov Radiation Simulation")
clock = pygame.time.Clock()

# Materials with refractive indices
materials = [
    {"name": "Fused Silica", "n": 1.46, "color": (200, 200, 255, 100)},
    {"name": "Aerogel 1", "n": 1.03, "color": (220, 220, 255, 100)},
    {"name": "Aerogel 2", "n": 1.05, "color": (210, 210, 255, 100)},
    {"name": "Acrylic", "n": 1.49, "color": (190, 190, 255, 100)},
    {"name": "CYTOP", "n": 1.34, "color": (180, 180, 255, 100)},
    {"name": "Sapphire", "n": 1.77, "color": (170, 170, 255, 100)}
]

# Particle types
particles = [
    {"name": "Electron", "mass": 0.511, "color": BLUE},
    {"name": "Pion", "mass": 139.6, "color": RED},
    {"name": "Proton", "mass": 938.3, "color": YELLOW}
]

# Current simulation settings
current_material = 0
current_particle = 0
current_momentum = 5  # GeV/c
current_angle = 0  # degrees

# Particle beam
particles_in_beam = []
cherenkov_photons = []
cherenkov_rings = []  # Store complete rings for visualization

# Calculate beta (v/c) from momentum and particle mass
def calculate_beta(momentum, mass):
    energy = math.sqrt((momentum * 1000)**2 + mass**2)  # Convert GeV to MeV
    return momentum * 1000 / energy

# Calculate Cherenkov angle
def calculate_cherenkov_angle(n, beta):
    if beta * n <= 1:
        return None  # Cherenkov radiation not produced
    return math.acos(1 / (n * beta))

# Fonts
font = pygame.font.SysFont('Arial', 16)
title_font = pygame.font.SysFont('Arial', 26, bold=True)
subtitle_font = pygame.font.SysFont('Arial', 20, bold=True)

# Main simulation area layout
side_panel_width = 220
side_margin = 20
detector_x = side_panel_width + side_margin*2
detector_width = WIDTH - detector_x - side_panel_width - side_margin*2
detector_height = HEIGHT - 180
detector_y = 90

# Ring imaging area
ring_image_x = WIDTH - side_panel_width // 2
ring_image_y = HEIGHT - 120
ring_image_radius = 80

# Wheel position and control variables
wheel_center = (side_panel_width // 2, 180)
wheel_radius = 80
material_angle = 0
rotating = False

# Create a grid pattern
grid_spacing = 50
grid_color = (40, 40, 60)

# Load or create background texture
def create_background_texture():
    texture = pygame.Surface((WIDTH, HEIGHT))
    texture.fill(DARK_GRAY)
    # Add some subtle noise
    for _ in range(5000):
        x = random.randint(0, WIDTH-1)
        y = random.randint(0, HEIGHT-1)
        brightness = random.randint(15, 35)
        texture.set_at((x, y), (brightness, brightness, brightness+5))
    return texture

background_texture = create_background_texture()

class Particle:
    def __init__(self, x, y, angle, momentum, particle_type):
        self.x = x
        self.y = y
        self.angle = math.radians(angle)
        self.momentum = momentum
        self.particle_type = particle_type
        self.beta = calculate_beta(momentum, particles[particle_type]["mass"])
        self.speed = 8  # pixels per frame
        self.active = True
        self.cherenkov_produced = False
        self.trail = []  # Store positions for particle trail
        self.max_trail_length = 15
        
    def update(self):
        # Add current position to trail
        self.trail.append((self.x, self.y))
        if len(self.trail) > self.max_trail_length:
            self.trail.pop(0)
            
        dx = self.speed * math.cos(self.angle)
        dy = self.speed * math.sin(self.angle)
        self.x += dx
        self.y += dy
        
        # Check if in detector
        if (detector_x < self.x < detector_x + detector_width and 
            detector_y < self.y < detector_y + detector_height and 
            not self.cherenkov_produced):
            
            # Calculate Cherenkov angle
            n = materials[current_material]["n"]
            theta_c = calculate_cherenkov_angle(n, self.beta)
            
            if theta_c is not None:
                # Create Cherenkov photons
                num_photons = int(40 * math.sin(theta_c)**2)  # More photons for larger angles
                
                # Also create a complete Cherenkov ring for visualization
                cherenkov_rings.append({
                    "x": self.x,
                    "y": self.y,
                    "angle": theta_c,
                    "particle_angle": self.angle,
                    "color": (100, 160, 255, 180),
                    "life": 140,  # Longer life for the ring
                    "max_radius": 250,  # Maximum size of the ring
                    "current_radius": 5,  # Start small and expand
                    "expansion_rate": 3  # How fast the ring expands
                })
                
                for _ in range(num_photons):
                    # Calculate photon direction (cone around particle direction)
                    phi = random.uniform(0, 2 * math.pi)
                    photon_angle = self.angle + math.pi/2
                    photon_x = self.x + 5 * math.cos(photon_angle)
                    photon_y = self.y + 5 * math.sin(photon_angle)
                    
                    # Create a photon on the Cherenkov cone
                    cone_vec_x = math.cos(self.angle) * math.cos(theta_c) + math.sin(self.angle) * math.sin(theta_c) * math.cos(phi)
                    cone_vec_y = math.sin(self.angle) * math.cos(theta_c) - math.cos(self.angle) * math.sin(theta_c) * math.cos(phi)
                    
                    cherenkov_photons.append({
                        "x": photon_x,
                        "y": photon_y,
                        "vx": cone_vec_x * 15,  # Photon speed
                        "vy": cone_vec_y * 15,
                        "life": 40,  # Increased photon life
                        "angle": theta_c
                    })
                
                self.cherenkov_produced = True
        
        # Check if out of bounds
        if self.x < 0 or self.x > WIDTH or self.y < 0 or self.y > HEIGHT:
            self.active = False

# Draw a ring with an aperture
def draw_ring_with_aperture(surface, color, center, radius, thickness, aperture_angle, rotation_angle):
    s = pygame.Surface((radius*2 + thickness, radius*2 + thickness), pygame.SRCALPHA)
    
    # Draw full ring
    pygame.draw.circle(s, color, (radius + thickness//2, radius + thickness//2), radius, thickness)
    
    # Clean screen
    screen.blit(s, (center[0] - radius - thickness//2, center[1] - radius - thickness//2))

# Draw a button
def draw_button(text, rect, active=False):
    color = (80, 80, 100) if not active else (100, 100, 180)
    border_color = LIGHT_BLUE if active else WHITE
    
    pygame.draw.rect(screen, color, rect, border_radius=5)
    pygame.draw.rect(screen, border_color, rect, 2, border_radius=5)
    
    text_surf = font.render(text, True, WHITE)
    text_rect = text_surf.get_rect(center=rect.center)
    screen.blit(text_surf, text_rect)
    
    return rect  # Return the rect for click detection

# Draw a panel with title
def draw_panel(rect, title, alpha=220):
    s = pygame.Surface((rect.width, rect.height), pygame.SRCALPHA)
    s.fill((30, 30, 40, alpha))
    screen.blit(s, rect)
    pygame.draw.rect(screen, WHITE, rect, 2, border_radius=5)
    
    if title:
        title_surf = subtitle_font.render(title, True, WHITE)
        screen.blit(title_surf, (rect.x + 10, rect.y + 5))
        # Draw separator line
        pygame.draw.line(screen, WHITE, 
                        (rect.x + 5, rect.y + 35), 
                        (rect.x + rect.width - 5, rect.y + 35),
                        1)

# Main simulation loop
running = True
auto_spawn = False
auto_spawn_timer = 0
auto_spawn_interval = 60  # Frames between auto-spawns
show_help = False

while running:
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_m:
                # Change material
                current_material = (current_material + 1) % len(materials)
            elif event.key == pygame.K_p:
                # Change particle
                current_particle = (current_particle + 1) % len(particles)
            elif event.key == pygame.K_UP:
                # Increase momentum
                current_momentum = min(10, current_momentum + 1)
            elif event.key == pygame.K_DOWN:
                # Decrease momentum
                current_momentum = max(1, current_momentum - 1)
            elif event.key == pygame.K_LEFT:
                # Change angle
                current_angle = (current_angle - 10) % 360
            elif event.key == pygame.K_RIGHT:
                # Change angle
                current_angle = (current_angle + 10) % 360
            elif event.key == pygame.K_SPACE:
                # Toggle wheel rotation
                rotating = not rotating
            elif event.key == pygame.K_a:
                # Toggle auto spawn
                auto_spawn = not auto_spawn
            elif event.key == pygame.K_c:
                # Clear all particles and rings
                particles_in_beam = []
                cherenkov_photons = []
                cherenkov_rings = []
            elif event.key == pygame.K_h:
                # Toggle help
                show_help = not show_help
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # Add a new particle to the beam
            if event.button == 1 and detector_x > 50:  # Left click
                new_particle = Particle(50, detector_y + detector_height//2, current_angle, current_momentum, current_particle)
                particles_in_beam.append(new_particle)
    
    # Auto spawn particles
    if auto_spawn:
        auto_spawn_timer += 1
        if auto_spawn_timer >= auto_spawn_interval:
            new_particle = Particle(50, detector_y + detector_height//2, current_angle, current_momentum, current_particle)
            particles_in_beam.append(new_particle)
            auto_spawn_timer = 0
    
    # Clear screen and draw background
    screen.blit(background_texture, (0, 0))
    
    # Draw grid
    for x in range(0, WIDTH, grid_spacing):
        pygame.draw.line(screen, grid_color, (x, 0), (x, HEIGHT))
    for y in range(0, HEIGHT, grid_spacing):
        pygame.draw.line(screen, grid_color, (0, y), (WIDTH, y))
    
    # Draw title bar
    title_rect = pygame.Rect(0, 0, WIDTH, 70)
    s = pygame.Surface((WIDTH, 70), pygame.SRCALPHA)
    s.fill((20, 20, 30, 230))
    screen.blit(s, (0, 0))
    pygame.draw.line(screen, LIGHT_BLUE, (0, 70), (WIDTH, 70), 2)
    
    # Title
    title_text = title_font.render("CHARM: Cherenkov Angle Resolution Measurements", True, WHITE)
    screen.blit(title_text, (WIDTH // 2 - title_text.get_width() // 2, 20))
    
    # Subtitle
    subtitle_text = font.render("Particle Physics Simulation of Cherenkov Radiation", True, LIGHT_BLUE)
    screen.blit(subtitle_text, (WIDTH // 2 - subtitle_text.get_width() // 2, 50))
    
    # Update material wheel angle
    if rotating:
        material_angle += 0.5
        if material_angle >= 360:
            material_angle = 0
        # Determine which material is active based on angle
        current_material = int((material_angle % 360) / 60)
    
    # Left panel - Material selection
    left_panel_rect = pygame.Rect(side_margin, 90, side_panel_width, 200)
    draw_panel(left_panel_rect, "Material Selection")
    
    # Draw material wheel
    pygame.draw.circle(screen, GRAY, wheel_center, wheel_radius)
    for i in range(6):
        angle = math.radians(i * 60 + material_angle)
        x = wheel_center[0] + wheel_radius * 0.7 * math.cos(angle)
        y = wheel_center[1] + wheel_radius * 0.7 * math.sin(angle)
        color = materials[i]["color"][:3] if i == current_material else (100, 100, 100)
        pygame.draw.circle(screen, color, (int(x), int(y)), 20)
        
        # Draw separator lines
        line_angle = math.radians(i * 60 + material_angle + 30)
        lx = wheel_center[0] + wheel_radius * math.cos(line_angle)
        ly = wheel_center[1] + wheel_radius * math.sin(line_angle)
        pygame.draw.line(screen, BLACK, wheel_center, (int(lx), int(ly)), 2)
    
    # Material info
    material_text = font.render(f"Current: {materials[current_material]['name']}", True, WHITE)
    screen.blit(material_text, (left_panel_rect.x + 10, left_panel_rect.y + 145))
    
    n_text = font.render(f"Refractive index: {materials[current_material]['n']}", True, WHITE)
    screen.blit(n_text, (left_panel_rect.x + 10, left_panel_rect.y + 170))
    
    # Left panel - Controls
    controls_rect = pygame.Rect(side_margin, 310, side_panel_width, 370)
    draw_panel(controls_rect, "Controls")
    
    controls_text = [
        "M: Change material",
        "P: Change particle",
        "UP/DOWN: Change momentum",
        "LEFT/RIGHT: Change angle",
        "SPACE: Rotate wheel",
        "A: Toggle auto-spawn",
        "C: Clear particles",
        "H: Toggle help",
        "LEFT CLICK: Add particle"
    ]
    
    for i, text in enumerate(controls_text):
        rendered = font.render(text, True, WHITE)
        screen.blit(rendered, (controls_rect.x + 10, controls_rect.y + 45 + i * 25))
    
    # Spawn point indicator
    pygame.draw.circle(screen, WHITE, (50, detector_y + detector_height//2), 5, 1)
    arrow_points = [
        (60, detector_y + detector_height//2),
        (75, detector_y + detector_height//2 - 5),
        (75, detector_y + detector_height//2 + 5)
    ]
    pygame.draw.polygon(screen, WHITE, arrow_points)
    
    # Draw semi-transparent detector area
    s = pygame.Surface((detector_width, detector_height), pygame.SRCALPHA)
    color = list(materials[current_material]["color"])
    color[3] = 80  # More transparent
    s.fill(color)
    screen.blit(s, (detector_x, detector_y))
    
    # Draw detector border with glowing effect
    for i in range(3):
        border_color = (LIGHT_BLUE[0], LIGHT_BLUE[1], LIGHT_BLUE[2], 100 - i*30)
        pygame.draw.rect(screen, border_color, 
                        (detector_x-i, detector_y-i, detector_width+i*2, detector_height+i*2), 
                        1, border_radius=3)
    
    # Right panel - Particle settings
    right_panel_rect = pygame.Rect(WIDTH - side_panel_width - side_margin, 90, side_panel_width, 200)
    draw_panel(right_panel_rect, "Particle Settings")
    
    # Particle info
    y_offset = right_panel_rect.y + 45
    text = font.render(f"Type: {particles[current_particle]['name']}", True, particles[current_particle]["color"])
    screen.blit(text, (right_panel_rect.x + 10, y_offset))
    
    y_offset += 30
    text = font.render(f"Momentum: {current_momentum} GeV/c", True, WHITE)
    screen.blit(text, (right_panel_rect.x + 10, y_offset))
    
    beta = calculate_beta(current_momentum, particles[current_particle]["mass"])
    y_offset += 30
    text = font.render(f"β (v/c): {beta:.3f}", True, WHITE)
    screen.blit(text, (right_panel_rect.x + 10, y_offset))
    
    y_offset += 30
    text = font.render(f"Angle: {current_angle}°", True, WHITE)
    screen.blit(text, (right_panel_rect.x + 10, y_offset))
    
    # Right panel - Cherenkov data
    cherenkov_rect = pygame.Rect(WIDTH - side_panel_width - side_margin, 310, side_panel_width, 230)
    draw_panel(cherenkov_rect, "Cherenkov Data")
    
    # Cherenkov info
    y_offset = cherenkov_rect.y + 45
    cherenkov_angle = calculate_cherenkov_angle(materials[current_material]["n"], beta)
    
    if cherenkov_angle is not None:
        angle_degrees = math.degrees(cherenkov_angle)
        text = font.render(f"Angle: {angle_degrees:.2f}°", True, BRIGHT_BLUE)
        
        # Draw small angle visualization
        angle_viz_x = cherenkov_rect.x + 110
        angle_viz_y = y_offset + 70
        pygame.draw.line(screen, WHITE, (angle_viz_x, angle_viz_y), 
                         (angle_viz_x + 70, angle_viz_y), 2)
        pygame.draw.line(screen, WHITE, (angle_viz_x, angle_viz_y), 
                         (angle_viz_x + 70 * math.cos(cherenkov_angle), 
                          angle_viz_y - 70 * math.sin(cherenkov_angle)), 2)
        pygame.draw.arc(screen, LIGHT_BLUE, 
                        (angle_viz_x - 15, angle_viz_y - 15, 30, 30), 
                        0, cherenkov_angle, 2)
    else:
        text = font.render("No Cherenkov radiation", True, RED)
    screen.blit(text, (cherenkov_rect.x + 10, y_offset))
    
    y_offset += 30
    if cherenkov_angle is not None:
        photon_yield = int(100 * math.sin(cherenkov_angle)**2)
        text = font.render(f"Photon Yield: {photon_yield}%", True, WHITE)
    else:
        text = font.render("Photon Yield: 0%", True, WHITE)
    screen.blit(text, (cherenkov_rect.x + 10, y_offset))
    
    # Ring imaging display
    img_panel_rect = pygame.Rect(WIDTH - side_panel_width - side_margin, 560, side_panel_width, 120)
    draw_panel(img_panel_rect, "Ring Imaging")
    
    # Draw ring display background
    pygame.draw.circle(screen, (40, 40, 50), (ring_image_x, ring_image_y), ring_image_radius)
    pygame.draw.circle(screen, WHITE, (ring_image_x, ring_image_y), ring_image_radius, 1)
    
    # Draw crosshairs
    pygame.draw.line(screen, (80, 80, 80), 
                    (ring_image_x - ring_image_radius, ring_image_y),
                    (ring_image_x + ring_image_radius, ring_image_y), 1)
    pygame.draw.line(screen, (80, 80, 80), 
                    (ring_image_x, ring_image_y - ring_image_radius),
                    (ring_image_x, ring_image_y + ring_image_radius), 1)
    
    # Draw Cherenkov ring in the imaging display
    if cherenkov_angle is not None:
        ring_radius = ring_image_radius * math.sin(cherenkov_angle)
        for i in range(3):
            pygame.draw.circle(screen, (BRIGHT_BLUE[0], BRIGHT_BLUE[1], BRIGHT_BLUE[2], 150 - i*40), 
                              (ring_image_x, ring_image_y), int(ring_radius) - i, 2)
    
    # Update and draw Cherenkov rings
    for ring in cherenkov_rings[:]:
        ring["current_radius"] += ring["expansion_rate"]
        ring["life"] -= 1
        
        if ring["life"] <= 0 or ring["current_radius"] > ring["max_radius"]:
            cherenkov_rings.remove(ring)
            continue
        
        # Calculate opacity based on life
        opacity = min(150, ring["life"] * 1.5)
        ring_color = (ring["color"][0], ring["color"][1], ring["color"][2], opacity)
        
        # Draw the ring at the origin point with appropriate thickness
        thickness = max(1, int(10 * (1 - ring["current_radius"] / ring["max_radius"])))
        draw_ring_with_aperture(
            screen, 
            ring_color, 
            (int(ring["x"]), int(ring["y"])),
            int(ring["current_radius"]),
            thickness,
            ring["angle"],
            ring["particle_angle"]
        )
    
    # Update and draw Cherenkov photons
    for photon in cherenkov_photons[:]:
        photon["x"] += photon["vx"]
        photon["y"] += photon["vy"]
        photon["life"] -= 1
        
        if photon["life"] <= 0:
            cherenkov_photons.remove(photon)
            continue
        
        # Cherenkov photons are blue with fading opacity
        opacity = min(255, photon["life"] * 6)
        photon_color = (100, 180, 255, opacity)
        s = pygame.Surface((6, 6), pygame.SRCALPHA)
        pygame.draw.circle(s, photon_color, (3, 3), 3)
        screen.blit(s, (int(photon["x"] - 3), int(photon["y"] - 3)))
    
    # Update and draw particles
    for particle in particles_in_beam[:]:
        particle.update()
        if not particle.active:
            particles_in_beam.remove(particle)
            continue
        
        # Draw particle trail
        for i, (trail_x, trail_y) in enumerate(particle.trail):
            alpha = int(255 * (i + 1) / len(particle.trail))
            trail_color = particles[particle.particle_type]["color"]
            s = pygame.Surface((4, 4), pygame.SRCALPHA)
            pygame.draw.circle(s, (*trail_color, alpha), (2, 2), 2)
            screen.blit(s, (int(trail_x - 2), int(trail_y - 2)))
        
        # Draw particle
        pygame.draw.circle(screen, particles[particle.particle_type]["color"], 
                         (int(particle.x), int(particle.y)), 5)
    
    # Status bar
    status_rect = pygame.Rect(0, HEIGHT - 30, WIDTH, 30)
    s = pygame.Surface((WIDTH, 30), pygame.SRCALPHA)
    s.fill((20, 20, 30, 200))
    screen.blit(s, (0, HEIGHT - 30))
    
    # Auto-spawn status
    if auto_spawn:
        auto_text = font.render("Auto-spawn: ON", True, BRIGHT_BLUE)
    else:
        auto_text = font.render("Auto-spawn: OFF", True, WHITE)
    screen.blit(auto_text, (10, HEIGHT - 25))
    
    # Display current particle count
    count_text = font.render(f"Particles: {len(particles_in_beam)} | Photons: {len(cherenkov_photons)} | Rings: {len(cherenkov_rings)}", True, WHITE)
    screen.blit(count_text, (WIDTH // 2 - count_text.get_width() // 2, HEIGHT - 25))
    
    # Help button
    help_text = font.render("Press H for Help", True, WHITE)
    screen.blit(help_text, (WIDTH - 120, HEIGHT - 25))
    
    # Show help overlay if needed
    if show_help:
        # Semi-transparent background
        s = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
        s.fill((0, 0, 20, 200))
        screen.blit(s, (0, 0))
        
        # Help panel
        help_width, help_height = 600, 500
        help_x = (WIDTH - help_width) // 2
        help_y = (HEIGHT - help_height) // 2
        help_rect = pygame.Rect(help_x, help_y, help_width, help_height)
        
        # Draw panel
        s = pygame.Surface((help_width, help_height), pygame.SRCALPHA)
        s.fill((40, 40, 60, 250))
        screen.blit(s, (help_x, help_y))
        pygame.draw.rect(screen, LIGHT_BLUE, help_rect, 2, border_radius=10)
        
        # Help title
        help_title = title_font.render("CHARM Experiment - Help", True, WHITE)
        screen.blit(help_title, (help_x + help_width//2 - help_title.get_width()//2, help_y + 20))
        
        # Separator
        pygame.draw.line(screen, LIGHT_BLUE, 
                         (help_x + 20, help_y + 60), 
                         (help_x + help_width - 20, help_y + 60), 2)
        
        # Help text
        help_lines = [
            "The CHARM experiment simulates Cherenkov radiation produced by high-energy",
            "particles moving through different materials.",
            "",
            "Cherenkov radiation occurs when a charged particle moves through a medium",
            "faster than light can travel in that medium. The characteristic blue glow is",
            "emitted in a cone shape around the particle's direction of travel.",
            "",
            "Experiment controls:",
            "- Use the LEFT MOUSE BUTTON to fire particles from the left side",
            "- Press M to cycle through different radiator materials",
            "- Press P to change particle type (electron, pion, proton)",
            "- UP/DOWN ARROWS change the particle momentum (1-10 GeV/c)",
            "- LEFT/RIGHT ARROWS change the particle angle",
            "- SPACE rotates the material wheel",
            "- A toggles automatic particle generation",
            "- C clears all particles and radiation",
            "",
            "The detector panel shows the material through which particles travel.",
            "The blue rings represent Cherenkov radiation cones.",
            "The side panels display physics information about the simulation.",
            "",
            "Press H to close this help window"
        ]
        
        for i, line in enumerate(help_lines):
            line_surf = font.render(line, True, WHITE)
            screen.blit(line_surf, (help_x + 30, help_y + 80 + i * 18))
    
    # Update display
    pygame.display.flip()
    clock.tick(FPS)

# Quit
pygame.quit()
sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-04-04 12:24:41.723 Python[4883:59311] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-04 12:24:41.723 Python[4883:59311] +[IMKInputSession subclass]: chose IMKInputSession_Modern


SystemExit: 

/Users/aayambansal/Desktop/VStudio/eurips/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 